<a href="https://colab.research.google.com/github/thehydrowave/Adversarial-diffusion-distillation-pokemon/blob/main/Addgood1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
!pip install numpy==1.26.4 diffusers==0.25.1 transformers==4.37.2 accelerate==0.27.2 \
datasets==2.17.0 huggingface_hub==0.25.0 peft==0.7.1 fsspec==2023.6.0 \
torch==2.2.2 torchvision==0.17.2 pandas==2.2.2 --no-cache-dir --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 123.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 278.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 177.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 170.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 148.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 222.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 246.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 219.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 299.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from diffusers import StableDiffusionPipeline
from diffusers.models.attention_processor import LoRAAttnProcessor
from transformers import CLIPTokenizer
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
from torchvision.io import read_image
from datasets import Dataset
import os
import torchvision.transforms as transforms
from diffusers import StableDiffusionPipeline


print("✅ NumPy:", np.__version__)

/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

✅ NumPy: 1.26.4


/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
# 📥 Charger le dataset Naruto
dataset = load_dataset("lambdalabs/naruto-blip-captions", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
# Créer un dossier temporaire
os.makedirs("naruto_images", exist_ok=True)

processed = []

for i, example in enumerate(tqdm(dataset)):
    # Resize et sauvegarde de l'image
    image = example["image"].convert("RGB").resize((64, 64))  # Resize to 256x256
    image_path = f"naruto_images/img_{i}.png"
    image.save(image_path)

    processed.append({
        "image_path": image_path,
        "text": example["text"]
    })

# Créer un dataset Hugging Face basé sur les chemins
dataset_small = Dataset.from_list(processed)

from torchvision.io import read_image

def transform_from_path(example):
    image_tensor = read_image(example["image_path"]).float() / 255.0
    example["pixel_values"] = image_tensor
    return example

dataset_small = dataset_small.map(transform_from_path)

100%|██████████| 1221/1221 [00:45<00:00, 26.75it/s]


Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
# 🔧 Charger le modèle SD + tokenizer
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

tokenizer = pipe.tokenizer


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
dataset_small = dataset_small.select(range(500))
# 📦 Créer un DataLoader
def collate_fn(batch):
    # Ensure pixel_values are tensors before stacking
    images = torch.stack([torch.tensor(example["pixel_values"]) if isinstance(example["pixel_values"], list) else example["pixel_values"] for example in batch])
    prompts = [example["text"] for example in batch]
    inputs = tokenizer(prompts, padding="max_length", truncation=True, return_tensors="pt")
    return {
        "pixel_values": images,
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
    }

# Use dataset_small instead of dataset
train_dataloader = DataLoader(dataset_small, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [ ]:
from diffusers.models.attention_processor import LoRAAttnProcessor

# 1. Récupérer la config des dimensions des layers
cross_attention_dim = pipe.unet.config.cross_attention_dim
attention_dims = {
    "mid_block": pipe.unet.config.block_out_channels[-1],
    "up_blocks.3": pipe.unet.config.block_out_channels[0],
    "up_blocks.2": pipe.unet.config.block_out_channels[1],
    "up_blocks.1": pipe.unet.config.block_out_channels[2],
    "up_blocks.0": pipe.unet.config.block_out_channels[3],
    "down_blocks.0": pipe.unet.config.block_out_channels[0],
    "down_blocks.1": pipe.unet.config.block_out_channels[1],
    "down_blocks.2": pipe.unet.config.block_out_channels[2],
    "down_blocks.3": pipe.unet.config.block_out_channels[3],
}

# 2. Injecter les LoRA à la main
attn_processors = {}
for name, processor in pipe.unet.attn_processors.items():
    for block_key in attention_dims:
        if name.startswith(block_key):
            hidden_size = attention_dims[block_key]
            is_cross_attention = name.endswith("attn2.processor")

            attn_processors[name] = LoRAAttnProcessor(
                hidden_size=hidden_size,
                cross_attention_dim=cross_attention_dim if is_cross_attention else None
            )
            break
    else:
        # Ne pas modifier les autres
        attn_processors[name] = processor

pipe.unet.set_attn_processor(attn_processors)
print(f"✅ LoRA manuellement appliqué à {sum(isinstance(p, LoRAAttnProcessor) for p in attn_processors.values())} couches.")

# 3. Récupérer les poids LoRA entraînables
lora_params = []
for name, processor in pipe.unet.attn_processors.items():
    if isinstance(processor, LoRAAttnProcessor):
        for param in processor.parameters():
            param.requires_grad = True
            lora_params.append(param)

if not lora_params:
    raise ValueError("❌ Toujours aucun poids LoRA détecté.")
else:
    print(f"🧠 Nombre total de paramètres LoRA : {sum(p.numel() for p in lora_params)}")

# 4. Optimiseur
optimizer = torch.optim.Adam(lora_params, lr=1e-4)



✅ LoRA manuellement appliqué à 0 couches.
🧠 Nombre total de paramètres LoRA : 797184


In [ ]:
from diffusers.models.attention_processor import LoRAAttnProcessor

# 1. Activer LoRA
pipe.enable_lora()
print(f"✅ LoRA activé pour {len(pipe.unet.attn_processors)} couches.")

# 2. ⚡ Warm-up réel du UNet pour créer les poids LoRA internes
pipe.unet.train()  # active les couches avec gradients

# on simule un passage dans le modèle pour activer les modules LoRA
dummy_latents = torch.randn(1, 4, 64, 64).to(pipe.device, dtype=torch.float16)
dummy_timestep = torch.tensor([1], device=pipe.device)
dummy_embedding = torch.randn(1, 77, pipe.unet.config.cross_attention_dim).to(pipe.device, dtype=torch.float16)

with torch.no_grad():
    _ = pipe.unet(dummy_latents, dummy_timestep, encoder_hidden_states=dummy_embedding)

# 3. Collecte des poids LoRA entraînables
lora_params = []
for name, processor in pipe.unet.attn_processors.items():
    if isinstance(processor, LoRAAttnProcessor):
        for param in processor.parameters():
            param.requires_grad = True
            lora_params.append(param)

# 4. Vérification
if not lora_params:
    raise ValueError("❌ Aucun poids LoRA détecté.")
else:
    print(f"🧠 Nombre total de paramètres LoRA entraînables : {sum(p.numel() for p in lora_params)}")

# 5. Optimiseur
optimizer = torch.optim.Adam(lora_params, lr=1e-4)





✅ LoRA activé pour 32 couches.


ValueError: ❌ Aucun poids LoRA détecté.

In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
# 🔁 Boucle d'entraînement
gradient_accumulation_steps = 4  # Define the variable before using it
for epoch in range(1):
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch}")
    for step, batch in enumerate(loop):
        pixel_values = batch["pixel_values"].type(torch.float16).to("cuda")
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")

        noise = torch.randn_like(pixel_values)
        noised_images = pixel_values + noise

        timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (pixel_values.shape[0],)).long().to("cuda").type(torch.float16)
        encoder_hidden_states = pipe.text_encoder(input_ids.to("cuda"))[0].type(torch.float16)

        noised_images = torch.cat([noised_images, torch.zeros_like(noised_images[:, :1, :, :])], dim=1)

        for param in pipe.unet.parameters():
            param.data = param.data.type(torch.float16)

        #with torch.cuda.amp.autocast(dtype=torch.float16):
            #model_output = pipe.unet(noised_images, timesteps, encoder_hidden_states).sample
            #loss = model_output.mean()

        model_output = pipe.unet(noised_images, timesteps, encoder_hidden_states).sample
        loss = model_output.mean()
        loss = loss / gradient_accumulation_steps  # Accumulate gradients
        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
           optimizer.step()
           optimizer.zero_grad()

        loop.set_postfix(loss=loss.item())

Epoch 0:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/diffusers/models/attention_processor.py:1871: FutureWarning: `LoRAAttnProcessor` is deprecated and will be removed in version 0.26.0. Make sure use AttnProcessor instead by settingLoRA layers to `self.{to_q,to_k,to_v,to_out[0]}.lora_layer` respectively. This will be done automatically when using `LoraLoaderMixin.load_lora_weights`
  deprecate(
Epoch 0: 100%|██████████| 500/500 [02:00<00:00,  4.16it/s, loss=0.000325]


In [ ]:
# ✅ Sauvegarde LoRA au format standard Hugging Face
output_dir = "./naruto_lora_weights"
os.makedirs(output_dir, exist_ok=True)

pipe.unet.save_attn_processors(output_dir)
print(f"✅ Poids LoRA sauvegardés dans : {output_dir}")


AttributeError: 'UNet2DConditionModel' object has no attribute 'save_attn_processors'

In [ ]:
output_dir = "./naruto_lora_weights"
os.makedirs(output_dir, exist_ok=True)

# 🔄 On rassemble tous les poids LoRA des attn_processors
lora_state_dict = {}
for name, processor in pipe.unet.attn_processors.items():
    if isinstance(processor, LoRAAttnProcessor):
        for k, v in processor.state_dict().items():
            lora_state_dict[f"{name}.{k}"] = v

# 💾 Sauvegarde dans un seul fichier
torch.save(lora_state_dict, os.path.join(output_dir, "pytorch_lora_weights.bin"))
print("✅ Poids LoRA sauvegardés dans :", os.path.join(output_dir, "pytorch_lora_weights.bin"))


✅ Poids LoRA sauvegardés dans : ./naruto_lora_weights/pytorch_lora_weights.bin
